# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [42]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [43]:
# Read the Mapbox API key
load_dotenv("C:\\Users\\rhnil\\Desktop\\Columbia_Bootcamp\\mapbox.env")
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [44]:
# Import the necessary CSVs to Pandas DataFrames

sf_neighborhood_locations = pd.read_csv(Path("./Data/neighborhoods_coordinates.csv"),index_col = 'Neighborhood')
sf_neighborhood_locations.index.rename("neighborhood", inplace = True) 
sf_neighborhood_locations.head()

,Lat,Lon
neighborhood,,
Alamo Square,37.791012,-122.402100
Anza Vista,37.779598,-122.443451
Bayview,37.734670,-122.401060
Bayview Heights,37.728740,-122.410980
Bernal Heights,37.728630,-122.443050


In [45]:
sf_data = pd.read_csv(Path("./Data/sfo_neighborhoods_census_data.csv"),index_col = 'year')
sf_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


In [46]:
sf_data_avg = sf_data.groupby([sf_data['neighborhood']]).mean()
sf_data_avg.head()

,sale_price_sqr_foot,housing_units,gross_rent
neighborhood,,,
Alamo Square,366.020712,378401.0,2817.285714
Anza Vista,373.382198,379050.0,3031.833333
Bayview,204.588623,376454.0,2318.400000
Bayview Heights,590.792839,382295.0,3739.000000
Bernal Heights,576.746488,379374.5,3080.333333


In [47]:
# Join dataframes to create full set of data for visualizations
sf_neighborhood_data_all = pd.concat([sf_data_avg, sf_neighborhood_locations], axis="columns", join="inner")
sf_neighborhood_data_all.reset_index(inplace=True)
sf_neighborhood_data_all.tail()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent,Lat,Lon
64,West Portal,498.488485,376940.75,2515.500000,37.74026,-122.463880
65,Western Addition,307.562201,377427.50,2555.166667,37.79298,-122.435790
66,Westwood Highlands,533.703935,376454.00,2250.500000,37.73470,-122.456854
67,Westwood Park,687.087575,382295.00,3959.000000,37.73415,-122.457000
68,Yerba Buena,576.709848,377427.50,2555.166667,37.79298,-122.396360


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [48]:
sf_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


In [87]:
# Construct datasets to be plotted
sf_avg_housing_units = sf_data["housing_units"].groupby([sf_data.index]).mean()

sf_avg_gross_rent = sf_data["gross_rent"].groupby([sf_data.index]).mean()

sf_avg_psf = sf_data["sale_price_sqr_foot"].groupby([sf_data.index]).mean()

sf_avg_psf_neighborhood = sf_data["sale_price_sqr_foot"].groupby([sf_data.neighborhood,sf_data.index]).mean()
sf_avg_psf_neighborhood_df = sf_avg_psf_neighborhood.to_frame()
sf_avg_psf_neighborhood_df.reset_index(inplace=True)
sf_avg_psf_neighborhood_df = sf_avg_psf_neighborhood_df[['year', 'neighborhood', 'sale_price_sqr_foot']]

psf_2016 = sf_avg_psf_neighborhood_df[sf_avg_psf_neighborhood_df.year == 2016]
top10_psf_2016 = psf_2016.sort_values(by='sale_price_sqr_foot',ascending=False).head(10)

sf_buy = sf_data.hvplot.line(x='year', y='sale_price_sqr_foot', groupby='neighborhood')
sf_rent = sf_data.hvplot.line(x='year', y='gross_rent', groupby='neighborhood')



In [84]:
# Define Panel Visualization Functions
def housing_units_per_year():
    bar_plot_height_base = sf_avg_housing_units.min()
    bar_plot_height_top = sf_avg_housing_units.max()
    yaxis_adj = sf_avg_housing_units.std()*2
    sf_housing_units_per_year_plot = sf_avg_housing_units.hvplot(y="housing_units", kind="bar", height=200, width=400, 
                             ylim=(bar_plot_height_base-yaxis_adj,bar_plot_height_top+yaxis_adj),
                             yformatter='%0u',
    )
    return sf_housing_units_per_year_plot


def average_gross_rent():
    sf_avg_gross_rent_plot = sf_avg_gross_rent.hvplot(x="year", y="gross_rent", kind="line", 
                                                      height=200, width=400
    )
    return sf_avg_gross_rent_plot
    


def average_sales_price():
    sf_avg_psf_plot = sf_avg_psf.hvplot(x="year", y="sale_price_sqr_foot", kind="line", height=200, width=400)
    return sf_avg_psf_plot



def average_price_by_neighborhood():
    sf_avg_psf_neighborhood_plot =sf_avg_psf_neighborhood_df.hvplot.line('year','sale_price_sqr_foot', groupby='neighborhood')
    return sf_avg_psf_neighborhood_plot


def top_most_expensive_neighborhoods():
    min = top10_psf_2016['sale_price_sqr_foot'].min()
    max = top10_psf_2016['sale_price_sqr_foot'].max()
    top10_psf_2016_plot = top10_psf_2016.hvplot.bar(x="neighborhood", y="sale_price_sqr_foot",  height=400, width=800, 
                             ylim=(0,max+100),
                             yformatter='%0u',
                             rot = 60
                             )
    return top10_psf_2016_plot


def most_expensive_neighborhoods_rent_sales():
    plots_as_column = pn.Column("Rent vs. Buy", sf_buy, sf_rent)
    plots_as_column  
    
    # YOUR CODE HERE!

    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    # YOUR CODE HERE!



def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!



def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!


def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
# Create panels to structure the layout of the dashboard
#geo_column = pn.Column("## Population and Crime Geo Plots", population_plot, crime_plot)
#scatter_column = pn.Column(
#    "## Correlation of Population and Crime Plots", population_violence, violent_murder
#)

# Create a tab layout for the dashboard
# YOUR CODE HERE!


# Create the dashboard
# YOUR CODE HERE!

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [51]:
housing_units_per_year()

:Bars   [year]   (housing_units)

In [63]:
average_gross_rent()

:Curve   [year]   (gross_rent)

In [71]:
average_sales_price()

:Curve   [year]   (sale_price_sqr_foot)

In [78]:
average_price_by_neighborhood()

:DynamicMap   [neighborhood]
   :Curve   [year]   (sale_price_sqr_foot)

In [85]:
top_most_expensive_neighborhoods()

:Bars   [neighborhood]   (sale_price_sqr_foot)

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()